In [ ]:
import pandas as pd
import numpy as np
import datetime

def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path)
        print("\nDataset loaded successfully!")
        return df
    except FileNotFoundError:
        print("File not found. Please check the file path.")
        return None

def explore_dataset(df):
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
    print("\nDataset Info:")
    print(df.info())
    print("\nSummary Statistics:")
    print(df.describe(include='all'))
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nDuplicate Rows:")
    print(df.duplicated().sum())

def validate_data(df):
    print("\nRunning basic data validation checks...")
    issues = []

    if 'AGE' in df.columns and pd.api.types.is_numeric_dtype(df['AGE']):
        try:
            if (df['AGE'] < 0).any():
                issues.append("AGE column contains negative values.")
        except TypeError:
            issues.append("AGE column contains non-numeric or incompatible values.")

    if 'ID' in df.columns:
        if df['ID'].duplicated().any():
            issues.append("ID column contains duplicate values.")

    null_counts = df.isnull().sum()
    if null_counts.any():
        issues.append("Some columns still have missing values:")
        issues.append(str(null_counts[null_counts > 0]))

    if issues:
        print("\nValidation Issues Found:")
        for issue in issues:
            print("-", issue)
    else:
        print("\nNo data validation issues found.")

def clean_dataset(df):
    df.columns = df.columns.str.upper()
    df = df.drop_duplicates().reset_index(drop=True)

    date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    df['__row_state__'] = 'valid'

    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip().str.lower().str.title()

    if 'ITEM' in df.columns:
        df['ITEM'] = df['ITEM'].replace(['Unknown', 'Error', '', 'Nan'], 'Other')

    for col in ['PAYMENT METHOD', 'LOCATION']:
        if col in df.columns:
            df[col] = df[col].replace(['Unknown', 'Error', '', 'Nan'], 'Other')

    if {'PRICE', 'QUANTITY', 'TOTAL SPENT'}.issubset(df.columns):
        df['PRICE'] = pd.to_numeric(df['PRICE'], errors='coerce')
        df['QUANTITY'] = pd.to_numeric(df['QUANTITY'], errors='coerce')
        df['TOTAL SPENT'] = pd.to_numeric(df['TOTAL SPENT'], errors='coerce')

        for idx, row in df.iterrows():
            p = row['PRICE']
            q = row['QUANTITY']
            t = row['TOTAL SPENT']

            known = [pd.notna(p), pd.notna(q), pd.notna(t)]

            if known.count(True) == 2:
                if pd.isna(p):  # Calculate PRICE if missing
                    if q != 0:
                        df.at[idx, 'PRICE'] = t / q
                    else:
                        df.at[idx, 'PRICE'] = 0
                elif pd.isna(q):  # Calculate QUANTITY if missing
                    if p != 0:
                        df.at[idx, 'QUANTITY'] = t / p
                    else:
                        df.at[idx, 'QUANTITY'] = 0
                elif pd.isna(t):  # Calculate TOTAL SPENT if missing
                    df.at[idx, 'TOTAL SPENT'] = p * q
            elif known.count(True) < 2:  # Set values to 0 if two or more columns are missing
                if pd.isna(p):
                    df.at[idx, 'PRICE'] = 0
                if pd.isna(q):
                    df.at[idx, 'QUANTITY'] = 0
                if pd.isna(t):
                    df.at[idx, 'TOTAL SPENT'] = 0

    for col in df.select_dtypes(include='object').columns:
        if col not in date_cols:
            df[col] = df[col].replace(['Unknown', 'Error', '', 'Nan'], 'Unknown')
            df[col] = df[col].fillna('Unknown')

    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[col] = df[col].fillna(df[col].median())

    df = df.loc[:, df.isnull().mean() < 0.5]

    if 'AGE' in df.columns:
        df['AGE'] = df['AGE'].round().astype('Int64')

    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d')
        missing_date_mask = df[col].isnull()
        df.loc[missing_date_mask, '__row_state__'] = 'missing_date'
        df[col] = df[col].fillna(pd.to_datetime('2020-01-01'))

    non_date_cols = [col for col in df.columns if col not in date_cols + ['__row_state__']]
    unknown_mask = df[non_date_cols].isin(['Unknown']).any(axis=1)
    missing_mask = df[non_date_cols].isnull().any(axis=1)
    df.loc[unknown_mask | missing_mask, '__row_state__'] = 'replaced_unknown'

    df = df.drop(columns=['__row_state__'], errors='ignore')

    print("\nCleaned Data Summary:")
    print("Columns after cleaning:", df.columns.tolist())
    print("Shape after cleaning:", df.shape)

    return df

def suggest_data_types(df):
    print("\nColumn Type Suggestions:")
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].nunique() / len(df) < 0.5:
            print(f"- Consider converting '{col}' to 'category'")
        elif pd.api.types.is_float_dtype(df[col]) and (df[col] % 1 == 0).all():
            print(f"- Consider converting '{col}' to 'int'")

def show_high_correlations(df, threshold=0.8):
    print("\nHighly Correlated Numerical Features:")
    corr = df.select_dtypes(include=[np.number]).corr()
    high_corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    result = high_corr.stack().reset_index()
    result.columns = ['Feature1', 'Feature2', 'Correlation']
    strong = result[abs(result['Correlation']) > threshold]
    print(strong if not strong.empty else "No strong correlations found.")

def detect_outliers(df):
    print("\nOutlier Detection:")
    numeric_cols = df.select_dtypes(include=[np.number])
    for col in numeric_cols.columns:
        Q1 = numeric_cols[col].quantile(0.25)
        Q3 = numeric_cols[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((numeric_cols[col] < (Q1 - 1.5 * IQR)) | (numeric_cols[col] > (Q3 + 1.5 * IQR))).sum()
        print(f"- {col}: {outliers} outliers")

def check_duplicate_columns(df):
    print("\nChecking for duplicate columns...")
    duplicates = df.T[df.T.duplicated()].T
    if not duplicates.empty:
        print(f"Duplicate columns found: {list(duplicates.columns)}")
    else:
        print("No duplicate columns found.")

def save_cleaned_dataset(df, output_file):
    try:
        df.to_csv(output_file, index=False)
        print(f"\nCleaned dataset saved as '{output_file}'")
    except Exception as e:
        print(f"Failed to save file: {e}")

def generate_cleaning_report(df, output_path="cleaning_report.txt"):
    try:
        with open(output_path, 'w') as report:
            report.write("DATA CLEANING REPORT\n")
            report.write("====================\n\n")
            report.write(f"Report Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            report.write(f"Final Shape: {df.shape}\n")
            report.write("\nColumns:\n")
            for col in df.columns:
                report.write(f"- {col} ({df[col].dtype})\n")
            report.write("\nNotes:\n")
            report.write("- Duplicates removed.\n")
            report.write("- Column names standardized.\n")
            report.write("- Categorical text cleaned and errors normalized.\n")
            report.write("- Missing values handled (mode/median/default date used).\n")
            report.write("- Highly missing columns dropped (>50%).\n")
            report.write("- Row order preserved.\n")
        print(f"Cleaning report saved to '{output_path}'")
    except Exception as e:
        print(f"Failed to write cleaning report: {e}")

# Sample usage
if __name__ == "__main__":
    file_path = input("Enter the path to your CSV file: ")
    df = load_dataset(file_path)

    if df is not None:
        explore_dataset(df)
        df = clean_dataset(df)
        validate_data(df)
        suggest_data_types(df)
        show_high_correlations(df)
        detect_outliers(df)
        check_duplicate_columns(df)
        output_file_name = input("Enter the output file name for the cleaned dataset (e.g. cleaned_data.csv): ")
        save_cleaned_dataset(df, output_file_name)
        generate_cleaning_report(df)
        print("\nData cleaning and preparation completed successfully!")

In [4]:
import pandas as pd

def handle_missing_values(df):
    print("\nHandling missing values for QUANTITY, PRICE PER UNIT, and TOTAL SPENT...")

    # Ensure column names are cleaned from any leading or trailing spaces
    df.columns = df.columns.str.strip()

    # Check column names
    print("Column names in the dataset:", df.columns)

    # Iterate through rows to handle missing values
    for idx, row in df.iterrows():
        # Get the values for QUANTITY, PRICE PER UNIT, and TOTAL SPENT
        quantity = row['QUANTITY']
        price_per_unit = row['PRICE PER UNIT']
        total_spent = row['TOTAL SPENT']
        
        # Convert to numeric where possible, invalid parsing will result in NaN
        try:
            quantity = pd.to_numeric(quantity, errors='coerce')
            price_per_unit = pd.to_numeric(price_per_unit, errors='coerce')
            total_spent = pd.to_numeric(total_spent, errors='coerce')
        except ValueError:
            continue  # If there's a value error, we just skip that row (though it should not occur with 'coerce')

        # Check for "Unknown" values
        missing_values = 0
        if row['QUANTITY'] == "Unknown":
            missing_values += 1
        if row['PRICE PER UNIT'] == "Unknown":
            missing_values += 1
        if row['TOTAL SPENT'] == "Unknown":
            missing_values += 1

        # If only one value is missing, calculate it based on the other two
        if missing_values == 1:
            if row['QUANTITY'] == "Unknown" and price_per_unit != "Unknown" and total_spent != "Unknown":
                df.at[idx, 'QUANTITY'] = total_spent / price_per_unit
            elif row['PRICE PER UNIT'] == "Unknown" and quantity != "Unknown" and total_spent != "Unknown":
                df.at[idx, 'PRICE PER UNIT'] = total_spent / quantity
            elif row['TOTAL SPENT'] == "Unknown" and quantity != "Unknown" and price_per_unit != "Unknown":
                df.at[idx, 'TOTAL SPENT'] = price_per_unit * quantity
        
        # If more than one value is missing, set them to 0
        elif missing_values > 1:
            df.at[idx, 'QUANTITY'] = 0
            df.at[idx, 'PRICE PER UNIT'] = 0
            df.at[idx, 'TOTAL SPENT'] = 0
        
        # Convert the values in the relevant columns to whole numbers (integers)
        df.at[idx, 'QUANTITY'] = int(float(df.at[idx, 'QUANTITY'])) if pd.notna(df.at[idx, 'QUANTITY']) else 0
        df.at[idx, 'PRICE PER UNIT'] = int(float(df.at[idx, 'PRICE PER UNIT'])) if pd.notna(df.at[idx, 'PRICE PER UNIT']) else 0
        df.at[idx, 'TOTAL SPENT'] = int(float(df.at[idx, 'TOTAL SPENT'])) if pd.notna(df.at[idx, 'TOTAL SPENT']) else 0

    return df

if __name__ == "__main__":
    # Get the input file path from the user
    input_file_path = input("Enter the path to your CSV file: ")

    # Load the dataset
    try:
        df = pd.read_csv(input_file_path)
        print("\nDataset loaded successfully!")
        
        # Clean column names
        df.columns = df.columns.str.strip()

        # Handle missing values
        df = handle_missing_values(df)
        
        # Save the cleaned dataset
        output_file_name = input("Enter the output file name for the cleaned dataset (e.g. final_cleaned_data.csv): ")
        df.to_csv(output_file_name, index=False)
        print(f"\nCleaned dataset saved as '{output_file_name}'")
    
    except FileNotFoundError:
        print(f"Error: The file at '{input_file_path}' was not found.")


Enter the path to your CSV file:  clean_cafe_sales.csv



Dataset loaded successfully!

Handling missing values for QUANTITY, PRICE PER UNIT, and TOTAL SPENT...
Column names in the dataset: Index(['TRANSACTION ID', 'ITEM', 'QUANTITY', 'PRICE PER UNIT', 'TOTAL SPENT',
       'PAYMENT METHOD', 'LOCATION', 'TRANSACTION DATE'],
      dtype='object')


Enter the output file name for the cleaned dataset (e.g. final_cleaned_data.csv):  final_clean_cafe_sales.csv



Cleaned dataset saved as 'final_clean_cafe_sales.csv'
